In [1]:
#IMPORT STATEMENTS
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
!pip install -q datasets transformers peft
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
# from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification,default_data_collator,get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.6 MB/s eta 0:00:00


In [2]:
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-small",use_cache=True)
gpu_name = '/device:GPU:0'
# torch.device(gpu_name)
# model.to(gpu_name)
# with torch.device('cpu', 0):
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-small")
dataset = load_dataset("sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/116M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [3]:
#VARS
seed_value = 42

# 1. Set the seed for NumPy
np.random.seed(seed_value)

# 2. Set the seed for Python's built-in random module
import random
random.seed(seed_value)
torch.manual_seed(seed_value)
# 3. Set the seed for TensorFlow
tf.random.set_seed(seed_value)
BATCH_SIZE = 128
hidden_dim = 128
num_tokens = 30
num_epoch = 3
lrn = 2e-2
max_length = 300

In [4]:
#FUNCTIONS

def preprocess_function(examples):
    inputs = examples['sentence']
    # targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    # labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    # labels = labels["input_ids"]
    # labels[labels == tokenizer.pad_token_id] = -100
    # print(inputs)
    model_inputs["labels"] = examples['label']
    return model_inputs

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

In [5]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/67349 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True)

In [7]:
# with torch.device('cpu', 0):
#   tokenized_train = dataset['train'].map(tokenization, batched = True)
#   tokenized_train.set_format('pt',
#                             columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

#   tokenized_train = tf.data.Dataset.from_tensor_slices(tokenized_train[:])
#   tokenized_train = tokenized_train.batch(BATCH_SIZE).shuffle(1000)
#   tokenized_train = tokenized_train.map(order, num_parallel_calls=tf.data.AUTOTUNE)
# # # model(tokenized_train)
# # exe = tokenizer(dataset['train']['sentence'],padding=True,truncation=True,max_length = 512,return_tensors='pt')
# # # exe['']



train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=BATCH_SIZE, pin_memory=True)

In [8]:
# !pip install -q peft
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=num_tokens)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 123,906 || all params: 28,888,580 || trainable%: 0.42890997065276315


In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lrn)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epoch),
)

In [10]:
device = "cuda"
model = model.to(device)

for epoch in range(num_epoch):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 7/7 [00:03<00:00,  2.25it/s]


epoch=0: train_ppl=tensor(1.5986, device='cuda:0') train_epoch_loss=tensor(0.4691, device='cuda:0') eval_ppl=tensor(1.5197, device='cuda:0') eval_epoch_loss=tensor(0.4185, device='cuda:0')


100%|██████████| 7/7 [00:03<00:00,  2.29it/s]


epoch=1: train_ppl=tensor(1.4414, device='cuda:0') train_epoch_loss=tensor(0.3656, device='cuda:0') eval_ppl=tensor(1.4840, device='cuda:0') eval_epoch_loss=tensor(0.3948, device='cuda:0')


100%|██████████| 7/7 [00:03<00:00,  2.31it/s]

epoch=2: train_ppl=tensor(1.3975, device='cuda:0') train_epoch_loss=tensor(0.3347, device='cuda:0') eval_ppl=tensor(1.4580, device='cuda:0') eval_epoch_loss=tensor(0.3770, device='cuda:0')


In [11]:
correct = 0
total = 0
for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            x = outputs.logits
            x = torch.exp(x)
            x = x/torch.sum(x,1,keepdim=True)
            for i in range(x.size()[0]):
              if(x[i][0] < x[i][1] and batch['labels'][i] == 1):
                correct += 1
              elif(x[i][0] > x[i][1] and batch['labels'][i] == 0):
                correct += 1
              total += 1
            # print(batch['labels'])
            # break
print("\n the accuracy is:-")
print(correct * 100 /total)

100%|██████████| 7/7 [00:02<00:00,  2.34it/s]


 the accuracy is:-
82.91284403669725
